In [1]:
import numpy as np
import pandas as pd

import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import optuna
import xgboost as xgb

from sklearn.metrics import r2_score

# Dataset Loading

In [2]:
data = pd.read_csv('D:\Solar Radiation Prediction\Dataset\SolarPrediction.csv')

In [3]:
data

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00
32682,1480587301,12/1/2016 12:00:00 AM,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00
32683,1480587001,12/1/2016 12:00:00 AM,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00
32684,1480586702,12/1/2016 12:00:00 AM,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00


In [4]:
data['Month'] = data['Data'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(int)
data['Day'] = data['Data'].apply(lambda x: re.search(r'(?<=\/)\d+(?=\/)', x).group(0)).astype(int)
data['Year'] = data['Data'].apply(lambda x: re.search(r'(?<=\/)\d+(?=\s)', x).group(0)).astype(int)

data = data.drop('Data', axis=1)

In [5]:
data['Hour'] = data['Time'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(int)
data['Minute'] = data['Time'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(int)
data['Second'] = data['Time'].apply(lambda x: re.search(r'\d+$', x).group(0)).astype(int)

data = data.drop('Time', axis=1)

In [6]:
data['SunriseHour'] = data['TimeSunRise'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(int)
data['SunriseMinute'] = data['TimeSunRise'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(int)

data['SunsetHour'] = data['TimeSunSet'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(int)
data['SunsetMinute'] = data['TimeSunSet'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(int)

data = data.drop(['TimeSunRise', 'TimeSunSet'], axis=1)

In [7]:
data = data.drop(['Year', 'SunriseHour'], axis=1)

In [8]:
y = data['Radiation'].copy()
X = data.drop('Radiation', axis=1).copy()

In [9]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=100)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=200)

# Linear Regression

In [11]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
y_true = np.array(y_test, dtype=float)
r2_linear = r2_score(y_true, y_pred)

# Decision Tree

In [12]:
from sklearn.tree import DecisionTreeRegressor 
regressor = DecisionTreeRegressor(random_state = 0) 
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
y_true = np.array(y_test, dtype=float)
r2_tree = r2_score(y_true, y_pred)

# Support Vector Regression

In [13]:
from sklearn.svm import SVR
svregressor = SVR(kernel = 'rbf')
svregressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
y_true = np.array(y_test, dtype=float)
r2_svr = r2_score(y_true, y_pred)

# Artificial Neural Network

In [14]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
 
model.add(Dense(units=50, input_dim=14, kernel_initializer='normal', activation='relu'))

model.add(Dense(units=50, kernel_initializer='normal', activation='tanh'))
 
model.add(Dense(1, kernel_initializer='normal'))
 
model.compile(loss='mean_squared_error', optimizer='adam')
 
model.fit(X_train, y_train, batch_size = 20, epochs = 50, verbose=1)

y_pred = model.predict(X_test)
y_true = np.array(y_test, dtype=float)

r2_ann = r2_score(y_true, y_pred)

Epoch 1/50
916/916 [==============================] - 1s 866us/step - loss: 131195.6250
Epoch 2/50
916/916 [==============================] - 1s 831us/step - loss: 116537.6328
Epoch 3/50
916/916 [==============================] - 1s 679us/step - loss: 105566.9922
Epoch 4/50
916/916 [==============================] - 1s 757us/step - loss: 97207.9609
Epoch 5/50
916/916 [==============================] - 1s 666us/step - loss: 91014.2500
Epoch 6/50
916/916 [==============================] - 1s 662us/step - loss: 86659.7266
Epoch 7/50
916/916 [==============================] - 1s 766us/step - loss: 83775.9219
Epoch 8/50
916/916 [==============================] - 1s 671us/step - loss: 82021.9766
Epoch 9/50
916/916 [==============================] - 1s 762us/step - loss: 81063.8125
Epoch 10/50
916/916 [==============================] - 1s 679us/step - loss: 80582.4375
Epoch 11/50
916/916 [==============================] - 1s 764us/step - loss: 80359.8828
Epoch 12/50
916/916 [=================

# Optuna (For Tuning Hyperparameters)

In [15]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [16]:
def get_model_rmse(params):
    mod = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'eval')], early_stopping_rounds=10, verbose_eval=0)
    results = mod.eval(dval)
    rmse = float(re.search(r'[\d.]+$', results).group(0))
    return rmse

In [17]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 0.00001, 10.0)
    max_depth = trial.suggest_int('max_depth', 4, 8)
    l1_reg = trial.suggest_loguniform('l1_reg', 0.00001, 10.0)
    l2_reg = trial.suggest_loguniform('l2_reg', 0.00001, 10.0)
    
    params = {'learning_rate': learning_rate, 'max_depth': max_depth, 'alpha': l1_reg, 'lambda': l2_reg}
    
    return get_model_rmse(params)

In [18]:
study = optuna.create_study()
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2022-01-02 18:09:25,283] A new study created in memory with name: no-name-55738730-1b1c-4de7-8610-6aa6468f29d3
C:\Users\Prajwal\anaconda3\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2022-01-02 18:09:27,212] Trial 0 finished with value: 93.191338 and parameters: {'learning_rate': 0.026990417326274923, 'max_depth': 8, 'l1_reg': 0.22427037289040827, 'l2_reg': 0.0018697823609213982}. Best is trial 0 with value: 93.191338.
[I 2022-01-02 18:09:29,028] Trial 1 finished with value: 184.741409 and parameters: {'learning_rate': 0.008706547624370013, 'max_depth': 8, 'l1_reg': 0.06793681851197442, 'l2_reg': 0.0004648746185824279}. Best is trial 0 with value: 93.191338.
[I 2022-01-02 18:09:29,217] Trial 2 finished with value: 19240212480.0 and parameters: {'learning_rate': 6.11494594973384, 'max_depth': 8, 'l1_reg': 0.00013967609851377112, 'l2_reg': 4.142281105573771}. Best is trial 0 with value: 93.191338.
[I 2022-01-02 18:09:29,637] Trial 3 finished with value: 101.213387 and parameters: {'learning_rate': 0.777234142826214, 'max_depth': 4, 'l1_reg': 0.04365932775317912, 'l2_reg': 0.037708914636973206}. Best is trial 0 with value: 93.191338.
[I 2022-01-02 18:09:31,132] Tri

In [19]:
best_params = study.best_params
best_params

{'learning_rate': 0.11124940270545293,
 'max_depth': 8,
 'l1_reg': 9.314397753503524,
 'l2_reg': 2.2584971571392447e-05}

# XGboost Model

In [20]:
mod = xgb.train(best_params, dtrain, num_boost_round=10000, evals=[(dval, 'eval')], early_stopping_rounds=10)

[18:11:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "l1_reg", "l2_reg" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	eval-rmse:343.08524
[1]	eval-rmse:309.20435
[2]	eval-rmse:279.41980
[3]	eval-rmse:253.31375
[4]	eval-rmse:230.46806
[5]	eval-rmse:210.29347
[6]	eval-rmse:192.69878
[7]	eval-rmse:177.38524
[8]	eval-rmse:164.25886
[9]	eval-rmse:152.74907
[10]	eval-rmse:143.07382
[11]	eval-rmse:134.54629
[12]	eval-rmse:127.17339
[13]	eval-rmse:120.94813
[14]	eval-rmse:115.79404
[15]	eval-rmse:111.21919
[16]	eval-rmse:107.46536
[17]	eval-rmse:104.33890
[18]	eval-rmse:101.63162
[19]	eval-rmse:99.57003
[20]	eval-rmse:97.45377
[21]	eval-rmse:95.54520
[22]	eval-rmse:94.37003
[23]	eval-rmse

In [21]:
y_true = np.array(y_test, dtype=float)
y_pred = np.array(mod.predict(dtest), dtype=float)
r2_XGB = r2_score(y_true, y_pred)

# Conclusion 

In [22]:
print("R^2 Score for Linear Regression: {:.4f}".format(r2_linear))
print("R^2 Score for Decision Tree: {:.4f}".format(r2_tree))
print("R^2 Score for SVM: {:.4f}".format(r2_svr))
print("R^2 Score for ANN: {:.4f}".format(r2_ann))
print("R^2 Score for XGboost & Optuna: {:.4f}".format(r2_XGB))

R^2 Score for Linear Regression: 0.6314
R^2 Score for Decision Tree: 0.8909
R^2 Score for SVM: 0.8909
R^2 Score for ANN: 0.8528
R^2 Score for XGboost & Optuna: 0.9407
